In [34]:
import requests
import sqlite3
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime 

In [55]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = 'Countries_by_GDP.csv'
table_attribs = ["Country", "GDP_USD_millions"]
df = pd.DataFrame(columns=table_attribs)

def extract(url, table_attribs):
    #extract the required information from the website and save it to a dataframe. 
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df

df = extract(df)
df

,Country,GDP_USD_millions
0,United States,"26,854,599"
1,China,"19,373,586"
2,Japan,"4,409,738"
3,Germany,"4,308,854"
4,India,"3,736,882"
...,...,...
186,Marshall Islands,291
187,Palau,262
188,Kiribati,248
189,Nauru,151


In [56]:
def transform(data): 
    #Converts the GDP information
    data['GDP_USD_millions'] = data['GDP_USD_millions'].str.replace(',', '')
    data['GDP_USD_millions'] = round(data['GDP_USD_millions'].astype(float) / 1000 , 2)
      
    data = data.rename(columns={'GDP_USD_millions': 'GDP_USD_billions'})
     
    return data 

In [57]:
df = transform(df)
df

,Country,GDP_USD_billions
0,United States,26854.60
1,China,19373.59
2,Japan,4409.74
3,Germany,4308.85
4,India,3736.88
...,...,...
186,Marshall Islands,0.29
187,Palau,0.26
188,Kiribati,0.25
189,Nauru,0.15


In [64]:
def load_to_csv( transformed_data, target_file): 
    transformed_data.to_csv(target_file) 
    

In [65]:
load_to_csv(df,csv_path)

In [60]:
def load_to_db(df,conn,table_name ):
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    

In [61]:
conn = sqlite3.connect(db_name)
load_to_db(df, conn, table_name)
conn.close()

In [66]:
query = f"SELECT * FROM {table_name}"

def run_query(query, conn):
    print(query)
    query_output = pd.read_sql(query, conn)
    print(query_output)

conn = sqlite3.connect(db_name)
run_query(query, conn)
conn.close()

SELECT * FROM Countries_by_GDP
              Country  GDP_USD_billions
0       United States          26854.60
1               China          19373.59
2               Japan           4409.74
3             Germany           4308.85
4               India           3736.88
..                ...               ...
186  Marshall Islands              0.29
187             Palau              0.26
188          Kiribati              0.25
189             Nauru              0.15
190            Tuvalu              0.06

[191 rows x 2 columns]


In [38]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./etl_project_log.txt","a") as f: 
        f.write(timestamp + ',' + message + '\n') 

In [52]:
log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df)

log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')
sql_connection = sqlite3.connect('World_Economies.db')

log_progress('SQL Connection initiated.')
load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')
query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')
sql_connection.close()